In [8]:
pip install opencv-contrib-python


  Using cached opencv_contrib_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.9.0.80-cp37-abi3-win_amd64.whl (45.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
from pytube import YouTube

os.makedirs("videos", exist_ok=True)

url = 'https://www.youtube.com/watch?v=kW_z-NMuZIU'

yt = YouTube(url)
stream = yt.streams.get_highest_resolution()
stream.download(output_path="videos")

'c:\\classes\\06_NLP_Preprocessing\\project\\videos\\[2023 AFC 카타르 아시안컵] 2023 AFC 카타르 아시안컵 호주 VS 대한민국 풀 하이라이트.mp4'

In [79]:
# 영상 캡처
import cv2

os.makedirs("vid_capture",exist_ok=True)

cap= cv2.VideoCapture(r"C:\classes\06_NLP_Preprocessing\project\videos\[2023 AFC 카타르 아시안컵] 2023 AFC 카타르 아시안컵 호주 VS 대한민국 풀 하이라이트.mp4")
cnt = 0 # 몇번째 frame인지 저장할 변수
print(cap.isOpened())
delay = int(50/FPS)  # 1000밀리초(1초)/FPS -> FPS 기준 1 Frame을 출력하는데 걸리는 시간

FPS = cap.get(cv2.CAP_PROP_FPS)

while cap.isOpened():
    succ, frame = cap.read() 
    if not succ : # 읽기 실패
        print('캡처 실패')
        break
    
    cv2.imshow('frame',frame)
    # 파일로 저장
    if cnt % 200 == 0 and (2400 < cnt < 23400):  
        cv2.imwrite(f"vid_capture/cap_{cnt+1}.jpg", frame)
    if cv2.waitKey(delay) == 27: # 키 입력을 1밀리초 대기(1/1000초). esc 입력시 종료
        break
    cnt+=1 

cap.release()
cv2.destroyAllWindows()


True


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [88]:
import cv2 
import numpy as np
import glob

os.makedirs("output/grayscale", exist_ok=True)
os.makedirs("output/move", exist_ok=True)
os.makedirs("output/bright", exist_ok=True)
os.makedirs("output/edge", exist_ok=True)
os.makedirs("output/red", exist_ok=True)
os.makedirs("output/yellow", exist_ok=True)
os.makedirs("output/red_yellow", exist_ok=True)


img_list = glob.glob('vid_capture/*.jpg')

cnt = 0
for img_path in img_list:
    fname = os.path.basename(img_path) # 파일명 조회
    fname, ext = os.path.splitext(fname)   # 파일이름, 확장자를 분리해서 변수에 저장.
    # 읽기
    img = cv2.imread(img_path)

    ## 이진화
    ### grayscale 변환. 
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img_bin = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
    save_path1 = f"output/grayscale/{fname}_binary_{cnt:03d}{ext}"  # 03d: 0으로 세자리 맞춤. 1->001
    cv2.imwrite(save_path1,  img_bin)
    
    ## 좌우 이동 (20px 이동)
    affine_matrix = np.array([[1, 0, 20], [0, 1, 0]], dtype="float32")
    img_move = cv2.warpAffine(img, affine_matrix, (0, 0))
    save_path2 = f"output/move/{fname}_move_{cnt:03d}{ext}"
    cv2.imwrite(save_path2, img_move)

    ## 밝기 변경 (50 만큼 밝게)
    img_bright = cv2.add(img, 50)
    save_path3 = f"output/bright/{fname}_bright_{cnt:03d}{ext}"
    cv2.imwrite(save_path3, img_bright)

    ## 엣지 검출
    img_edge = cv2.Canny(img_gray, 50, 150)
    save_path4 = f"output/edge/{fname}_edge_{cnt:03d}{ext}"
    cv2.imwrite(save_path4, img_edge)

    # 빨강색이 있는 영역 찾기 (대한민국)
    # 빨강색 범위 : 0~10, 170~180
    # 채도, 명도 범위 : 100~255 
    img_hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV) # BGR -> HSV
    low_red1 = np.array([0,100,100]) # 범위의 작은 지점 (색; 0, 채도 ;100, 명도; 100) 채도 명도는 동일하게
    up_red1 = np.array([15,255,255]) 
    low_red2 = np.array([170,100,100])
    up_red2 = np.array([180,255,255])

    r_mask1= cv2.inRange(img_hsv, low_red1, up_red1) # inRange 함수 ; 범위 해당 값은 255 / 나머지 다 0 변환 ; 이진화
    r_mask2 = cv2.inRange(img_hsv, low_red2, up_red2) 
    r_mask = r_mask1 + r_mask2 
    img_red = cv2.bitwise_and(img,img, mask=r_mask)
    save_path5 = f"output/red/{fname}_red_{cnt:03d}{ext}"
    cv2.imwrite(save_path5, img_red)

    # 노란색 영역 찾기 (호주)
    low_yell = np.array([20,100,100]) 
    up_yell = np.array([35,255,255]) 
    
    y_mask= cv2.inRange(img_hsv, low_yell, up_yell) # inRange 함수 ; 범위 해당 값은 255 / 나머지 다 0 변환 ; 이진화
    img_yell = cv2.bitwise_and(img,img, mask= y_mask)
    save_path6 = f"output/yellow/{fname}_yellow_{cnt:03d}{ext}"
    cv2.imwrite(save_path6, img_yell)

    # 빨간색 + 노란색

    mask=r_mask+y_mask
    img_red_yell = cv2.bitwise_and(img,img, mask= mask)
    save_path7 = f"output/red_yellow/{fname}_red_yellow_{cnt:03d}{ext}"
    cv2.imwrite(save_path7, img_red_yell)


    cnt += 1
    